# Project 2

## Setup 

Download the data from the worldbank

In [3]:
# download and update the data in the project
!./setup.sh

mkdir: cannot create directory ‘data’: File exists
--2024-09-26 15:54:58--  https://databank.worldbank.org/data/download/EdStats_CSV.zip
Resolving databank.worldbank.org (databank.worldbank.org)... 104.18.42.231, 172.64.145.25, 2606:4700:4400::ac40:9119, ...
Connecting to databank.worldbank.org (databank.worldbank.org)|104.18.42.231|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://databankfiles.worldbank.org/public/ddpext_download/EdStats_CSV.zip [following]
--2024-09-26 15:54:58--  https://databankfiles.worldbank.org/public/ddpext_download/EdStats_CSV.zip
Resolving databankfiles.worldbank.org (databankfiles.worldbank.org)... 13.107.246.56, 2620:1ec:bdf::56
Connecting to databankfiles.worldbank.org (databankfiles.worldbank.org)|13.107.246.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38943514 (37M) [application/x-zip-compressed]
Saving to: ‘EdStats_CSV.zip’

EdStats_CSV.zip     100%[===================>]  

In [4]:
!pip install -r requirements.txt

In [5]:
import pandas as pd
import scipy.stats as stats
from scipy.stats import shapiro
from scipy.stats import kruskal
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.api as sms

In [14]:
# load and clean the dataset
country_codes = pd.read_csv("./data/countries.csv")
df = pd.read_csv('./data/EdStatsData.csv')

Setup the python environment

In [15]:
hyp1_df = df[df["Country Code"].isin(country_codes["alpha-3"])]
hyp1_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1970,1971,1972,1973,1974,1975,...,2060,2065,2070,2075,2080,2085,2090,2095,2100,Unnamed: 69
91625,Afghanistan,AFG,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,NaN,NaN,NaN,NaN,7.05911,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91626,Afghanistan,AFG,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,NaN,NaN,NaN,NaN,2.53138,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91627,Afghanistan,AFG,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,NaN,NaN,NaN,NaN,0.22154,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91628,Afghanistan,AFG,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,NaN,NaN,NaN,NaN,11.42652,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91629,Afghanistan,AFG,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# define the Indicators from the dataset for the cleaning process of the data

# define the inidcators for the gpi
gov_exp_indicators = [
    "Expenditure on education as % of total government expenditure (%)"
]

# defin the inidicators for the drop out rate
wp_indicators_poorest = 'UIS: Total net attendance rate, lower secondary, poorest quintile, both sexes (%)'
qp_indicators_richest = 'UIS: Total net attendance rate, lower secondary, richest quintile, both sexes (%)'

    


In [17]:
# Re-shape the data to prevent the Years from being presented as individual columns
df_melted = hyp1_df.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
                    var_name="Year", value_name="Value")

df_melted.head()

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Afghanistan,AFG,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2,1970,NaN
1,Afghanistan,AFG,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.F,1970,NaN
2,Afghanistan,AFG,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.GPI,1970,NaN
3,Afghanistan,AFG,"Adjusted net enrolment rate, lower secondary, ...",UIS.NERA.2.M,1970,NaN
4,Afghanistan,AFG,"Adjusted net enrolment rate, primary, both sex...",SE.PRM.TENR,1970,NaN


In [20]:


# Convert the data to a numeric value to work with it
# df_melted['Year'] = pd.to_numeric(df_melted['Year'], errors='coerce')

# Get the list Gpi indicator values and drop out values
gpi_df = df_melted[df_melted['Indicator Name'].isin(gov_exp_indicators)]
WPI_df = df_melted[df_melted['Indicator Name'] == wp_indicators_poorest]
WPI_rich_df = df_melted[df_melted['Indicator Name'] == qp_indicators_richest]

# Drop the null values to clean the data
gpi_df_clean = gpi_df.dropna(subset=['Value'])
poorest_clean = WPI_df.dropna(subset=['Value'])
richest_clean = WPI_df.dropna(subset=['Value'])

# convert the values to numeric to work with them
gpi_df_clean['Value'] = pd.to_numeric(gpi_df_clean['Value'], errors='coerce')
richest_clean['Value'] = pd.to_numeric(richest_clean['Value'], errors='coerce')
poorest_clean['Value'] = pd.to_numeric(richest_clean['Value'], errors='coerce')
gpi_df_clean['WPI'] = poorest_clean['Value']/richest_clean['Value']

gpi_df_clean

/tmp/ipykernel_158826/391475126.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gpi_df_clean['Value'] = pd.to_numeric(gpi_df_clean['Value'], errors='coerce')
/tmp/ipykernel_158826/391475126.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  richest_clean['Value'] = pd.to_numeric(richest_clean['Value'], errors='coerce')
/tmp/ipykernel_158826/391475126.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value,WPI
7935949,Bangladesh,BGD,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,1980,5.263600,NaN
8108204,Ethiopia,ETH,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,1980,12.355540,NaN
8144854,Ghana,GHA,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,1980,7.641880,NaN
8558999,Sweden,SWE,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,1980,9.862020,NaN
8723924,Bangladesh,BGD,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,1981,6.634030,NaN
...,...,...,...,...,...,...,...
36116134,St. Lucia,LCA,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,2015,16.527161,NaN
36149119,Tajikistan,TJK,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,2015,16.276409,NaN
36163779,Togo,TGO,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,2015,17.985470,NaN
36204094,United Kingdom,GBR,Expenditure on education as % of total governm...,SE.XPD.TOTL.GB.ZS,2015,13.880240,NaN


In [55]:
# Prepare the first indicator

# The GPI for primary, secondary and upper secondary schools
gpi_values = hyp1_df['Value_GPI']
drop_out_values = hyp1_df["Value_Dropout"]

In [17]:
anova_result = stats.f_oneway(
    hyp1_df[hyp1_df["Value_GPI"] != "equal"]["Value_Dropout"],
    hyp1_df[hyp1_df['Value_GPI'] == "equal"]["Value_Dropout"],
)

print(f"f-ANOVA Statistic: {anova_result.statistic}")
print(f"P-value: {anova_result.pvalue}")

f-ANOVA Statistic: 2.2426012958722157
P-value: 0.1343038193795568


In [56]:
# Test for a correlation

correlation, p_value = stats.pearsonr(gpi_values, drop_out_values)

print(f"Pearsons correlation coefficient: {correlation}")
print(f"P-value: {p_value}")

Pearsons correlation coefficient: -0.20755224109799786
P-value: 8.300701558340677e-64


In [57]:
if p_value < 0.05:
    print("There is a statistically significant correlation between GPI and Drop-out rate (Reject H0)")
else:
    print("There is no statistically significant correlation between GPI and Drop-out rate (Faile to reject H0)")

There is a statistically significant correlation between GPI and Drop-out rate (Reject H0)


In [16]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=gpi_values, y=drop_out_values)
plt.title('Gender Parity Index (GPI) vs Drop-out Rates')
plt.xlabel('Gender Parity Index (GPI)')
plt.ylabel('Drop-out Rates')
plt.show()

NameError: name 'gpi_values' is not defined

<Figure size 1000x600 with 0 Axes>

### Interpretation of results:

Pearson correlation coefficient: -0.25

This is a weak negative correlation between **gender parity (GPI)** and **Drop out rates**. This suggests that as GPI increases (.i.e. gender parity improves), the drop out rate decrease slightly but the effect is not very strong

P-Value: 1.7583200777245715e-114:

The p-value is extremely small, much smaller than the usual threshold of 0.05. This means the correlation is statistically significant. Even though the correlation is weak, the p-value indicates that the result is not due to random chance

### Conclusion:

We can confidently reject the null hypothesis (H0), meaning there is indeed a relationship between gender parity and drop out rates 


## Data analysis

In [59]:
print("Descriptive Statistics for GPI")
print(hyp1_df['Value_GPI'].describe())

print("\nDescriptive Statistics for Drop-out Rates")
print(hyp1_df['Value_Dropout'].describe())

Descriptive Statistics for GPI
count    6461.000000
mean        0.976505
std         0.182833
min         0.190660
25%         0.962490
50%         1.001210
75%         1.025080
max         2.494290
Name: Value_GPI, dtype: float64

Descriptive Statistics for Drop-out Rates
count    6461.000000
mean       19.195747
std        17.929322
min         0.007320
25%         4.069590
50%        13.937510
75%        30.442551
max        92.459167
Name: Value_Dropout, dtype: float64


In [60]:
median_dropout = hyp1_df['Value_Dropout'].median()
high_dropout = hyp1_df[hyp1_df['Value_Dropout'] > median_dropout]
low_dropout = hyp1_df[hyp1_df['Value_Dropout'] <= median_dropout]

In [61]:
# run t-testing
t_stat, p_value_ttest = stats.ttest_ind(high_dropout['Value_GPI'], low_dropout['Value_GPI'])
print(f"T-test: t-statistic = {t_stat}, p-value = {p_value_ttest}")

T-test: t-statistic = -13.973484421264791, p-value = 9.757703722627095e-44


In [ ]:
# do violin plot of groups

# test - for each group
# for normally distributed - shapiro - wilk test
# if normally - test variances -> levenes test
#                               -> bartlett test


# ALt for anova -> kruskal wallace test 
#               -> welsh anova 

In [62]:
gpi_values = hyp1_df['Value_GPI'].dropna()  # Drop missing values
dropout_values = hyp1_df['Value_Dropout'].dropna()  # Drop missing values

shapiro_gpi_stat, shapiro_gpi_pvalue = shapiro(gpi_values)
print(f"Shapiro-Wilk Test for GPI: Statistic = {shapiro_gpi_stat}, p-value = {shapiro_gpi_pvalue}")


shapiro_dropout_stat, shapiro_dropout_pvalue = shapiro(dropout_values)
print(f"Shapiro-Wilk Test for Drop-out Rates: Statistic = {shapiro_dropout_stat}, p-value = {shapiro_dropout_pvalue}")

if shapiro_gpi_pvalue < 0.05:
    print("GPI data is not normally distributed (Reject H0)")
else:
    print("GPI data is normally distributed (Fail to reject H0)")

if shapiro_dropout_pvalue < 0.05:
    print("Drop-out Rate data is not normally distributed (Reject H0)")
else:
    print("Drop-out Rate data is normally distributed (Fail to reject H0)")

Shapiro-Wilk Test for GPI: Statistic = 0.760653422998363, p-value = 1.4212061742204283e-70
Shapiro-Wilk Test for Drop-out Rates: Statistic = 0.8849735339081437, p-value = 1.996512272431664e-56
GPI data is not normally distributed (Reject H0)
Drop-out Rate data is not normally distributed (Reject H0)


/home/27728277/School/Datasci/27728277-rw771-Data_Science_Project_2/.venv/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 6461.
  res = hypotest_fun_out(*samples, **kwds)


In [64]:
# ANOVA test

country_groups_gpi = [group['Value_GPI'].dropna() for _, group in hyp1_df.groupby('Country Name')]
country_groups_dropout = [group['Value_Dropout'].dropna() for _, group in hyp1_df.groupby('Country Name')]

kruskal_gpi_stat, kruskal_gpi_pvalue = kruskal(*country_groups_gpi)
print(f"Kruskal-Wallis Test for GPI: Statistic = {kruskal_gpi_stat}, p-value = {kruskal_gpi_pvalue}")

kruskal_dropout_stat, kruskal_dropout_pvalue = kruskal(*country_groups_dropout)
print(f"Kruskal-Wallis Test for Drop-out Rates: Statistic = {kruskal_dropout_stat}, p-value = {kruskal_dropout_pvalue}")

if kruskal_gpi_pvalue < 0.05:
    print("There is a significant difference in GPI across countries (Reject H0)")
else:
    print("There is no significant difference in GPI across countries (Fail to reject H0)")

if kruskal_dropout_pvalue < 0.05:
    print("There is a significant difference in Drop-out Rates across countries (Reject H0)")
else:
    print("There is no significant difference in Drop-out Rates across countries (Fail to reject H0)")


Kruskal-Wallis Test for GPI: Statistic = 4161.903114806945, p-value = 0.0
Kruskal-Wallis Test for Drop-out Rates: Statistic = 4495.364696542772, p-value = 0.0
There is a significant difference in GPI across countries (Reject H0)
There is a significant difference in Drop-out Rates across countries (Reject H0)


#### Interpretation of ANOVA results



## Hypothesis 2 - Teacher training and student performance
### Hypothesis:
A higher percentage of trained teachers is postively correlated with student retention and lower drop-out rates
### Indicators:
1. Percentage of new recruited teachers trained (primary, secondary, by gender)
2. Cumulative drop-out rates

## Hypothesis 3 - School resources and Literacty:
### Hypothesis:
Schools with better access to infrastructure (electricty, potable water, toilets) will have higher enrolment rates

### Indicators:
1. Percentage of schools with access to electricity, potable water, mixed-sex/single-sex toilets
2. Net enrolment rates (primary, secondary)

## Hypothesis 4 - Government Expenditure and Enrolment

### Hypothesis:
Higher government expenditure on education correlates with increased enrolment rates

### Indicators:
1. Expenditure as % of total in public institutions (primary, secondary)
2. Net enrolement rate (primary, secondary)
